In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import re

In [2]:
# setting_dict = {
#     'Default': 'component_balance_False-add_transformer_False-add_LLM_TSFM_False-all_periods_False',
#     '+Resample': 'component_balance_True-add_transformer_False-add_LLM_TSFM_False-all_periods_False',
#     '+Transformer': 'component_balance_False-add_transformer_True-add_LLM_TSFM_False-all_periods_False',
#     '+AllPeriods': 'component_balance_False-add_transformer_False-add_LLM_TSFM_False-all_periods_True',
#     '+Resample+Transformer': 'component_balance_True-add_transformer_True-add_LLM_TSFM_False-all_periods_False',
#     '+Resample+AllPeriods': 'component_balance_True-add_transformer_False-add_LLM_TSFM_False-all_periods_True',
#     '+Resample+Transformer+AllPeriods': 'component_balance_True-add_transformer_False-add_LLM_TSFM_False-all_periods_True'
# }

setting_dict = {
    'Default': 'component_balance_False-add_transformer_True-add_LLM_TSFM_False-all_periods_False',
    '-Transformer': 'component_balance_False-add_transformer_False-add_LLM_TSFM_False-all_periods_False',
    '+Resample': 'component_balance_True-add_transformer_False-add_LLM_TSFM_False-all_periods_False',
    '+AllPL': 'component_balance_False-add_transformer_False-add_LLM_TSFM_False-all_periods_True',
}

In [3]:
metric = 'mse'
result_path = f'./meta/results'
datasets = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'traffic', 'weather', 'Exchange', 'ili']
datasets_vis = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Traffic', 'Weather', 'Exchange', 'ILI']

In [4]:
def df_generator(name, setting):
    # TSGym vs best sota
    dfs = []
    for pred_len_1, pred_len_2 in zip([96, 192, 336, 720], [24, 36, 48, 60]):
        file_list = [f'{dataset}-{setting}_{pred_len_1}_{pred_len_2}.npz' for dataset in datasets]
        df = {}
        for i, file in enumerate(file_list):
            # dataset = file.split('-')[0]
            dataset = file[:re.search('-component', file).start()]

            # 添加图例和标题
            perf_epoch = np.load(os.path.join(result_path, file), allow_pickle=True)
            top1_perf_epoch, top1_perf_epoch_mae = perf_epoch['top1_perf_epoch'], perf_epoch['top1_perf_epoch_mae']
            mse, mae = top1_perf_epoch[perf_epoch['best_epoch'].item()], top1_perf_epoch_mae[perf_epoch['best_epoch'].item()]

            df[dataset] = {'mse': mse, 'mae': mae}
        df = pd.DataFrame.from_dict(df).T
        # df.columns = [f'{_}_{name}' for _ in df.columns]
        dfs.append(df)
    dfs = pd.concat(dfs).groupby(level=0).mean()

    return dfs


In [5]:
df_ablation = {}
for k, v in setting_dict.items():
    try:
        df_ablation[k] = df_generator(k, v)
    except:
        pass
        continue
df_ablation = pd.concat(df_ablation, axis=1).round(3)
df_ablation = df_ablation.loc[datasets]
df_ablation.index = datasets_vis

In [6]:
df_ablation.to_excel('./meta/results_paper/TSGym-Ablation.xlsx')